# agent with Llamaindex


# Agent with LlamaIndex Notebook

This notebook demonstrates the implementation of a simple agent using LlamaIndex framework. It showcases:

- Setting up a local LLM model using Ollama
- Creating custom functions for multiplication and weather data retrieval
- Implementing an agent workflow with function tools
- Demonstrating both stateless and stateful agent interactions
- Using context objects to maintain conversation history

The agent is able to perform mathematical operations and fetch weather information through defined function tools.


In [2]:
pip install llama-index llama-index-llms-ollama

     |████████████████████████████████| 7.7 MB 1.0 MB/s eta 0:00:01     |███▊                            | 890 kB 2.1 MB/s eta 0:00:04
     |████████████████████████████████| 40 kB 808 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 720 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 455 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 172 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 912 kB/s eta 0:00:01
     |████████████████████████████████| 444 kB 2.2 M

In [17]:
pip install --upgrade typing_extensions # use python version >= 3.10 or update type extensions

You should consider upgrading via the '/Users/yohannesyimam/Documents/Projects/deffusion_model/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama

# initialize local model

In [6]:
llm = Ollama(model="dagbs/qwen2.5-coder-7b-instruct-abliterated:latest")

## functions

In [7]:
def multiply(a:int, b:int) -> int:
    """Multiply two numbers and return the result."""
    return a * b

In [39]:
import requests

def get_weather(city: str) -> str:
    """ a function that returns the weather data as a string value"""
    url = "https://api.weatherapi.com/v1/current.json"
    params = {"key": "e40acd9bb9154ea499d34832252505", "q": city, "aqi":"no"}

    resp = requests.get(url, params=params)
    if resp.status_code == 200:
        response_data = resp.json()
        return response_data["current"]["condition"]["text"]
    else:
        raise Exception("not data found")
    # return f"The weather in {city} is sunny today."

## agent

In [40]:
agent = AgentWorkflow.from_tools_or_functions(
    [FunctionTool.from_defaults(multiply),
     FunctionTool.from_defaults(get_weather,
                              name="my_weather_tool",
                              description="Useful for getting the weather for a given location.")],
    llm=llm
)

Agents are stateless by default, add remembering past interactions is opt-in using a Context object This might be useful if you want to use an agent that needs to remember previous interactions, like a chatbot that maintains context across multiple messages or a task manager that needs to track progress over time.

In [ ]:
# stateless
response = await agent.run("What is 2 times 3?")
print(response)
# remembering state
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("my name is john", ctx=ctx)
response = await agent.run("What was my name ", ctx=ctx)
print(response)


In [41]:
print(await agent.run("what is the weather in new york city?"))

It's currently overcast in New York City.
